<a href="https://colab.research.google.com/github/tousyou/keras/blob/master/criteo_datacolumn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [77]:
COLUMN_NAMES = ['Label']
for i in range(1,14):
  COLUMN_NAMES.append('I{0:d}'.format(i))
for i in range(1,27):
  COLUMN_NAMES.append('S{0:d}'.format(i))
print(COLUMN_NAMES)

LABEL_COLUMN = 'Label'
LABELS = [0, 1]
print(LABEL_COLUMN)

SELECT_COLUMNS = ['Label']
for i in range(1,14):
  SELECT_COLUMNS.append('I{0:d}'.format(i))
print(SELECT_COLUMNS)

['Label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26']
Label
['Label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13']


In [86]:
def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(file_path,
                                                 batch_size=120,
                                                 column_names=COLUMN_NAMES,
                                                 label_name=LABEL_COLUMN,
                                                 #select_columns=SELECT_COLUMNS,
                                                 field_delim='\t',
                                                 #shuffle=False,
                                                  num_epochs=1)
  return dataset

ds_train = get_dataset('./sample_data/criteo.txt')
ds_test = get_dataset('./sample_data/criteo.txt')

#examples, labels = next(iter(ds_train)) # 第一个批次
#print("EXAMPLES: \n", examples, "\n")
#print("LABELS: \n", labels)

In [79]:
numerical_columns = []
for i in range(1,len(SELECT_COLUMNS)):
  numerical_columns.append(tf.feature_column.numeric_column(SELECT_COLUMNS[i]))

print(numerical_columns)

category_columns = []
for i in range(1,27):
  item = tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_hash_bucket('S{0:d}'.format(i),200))
  category_columns.append(item)
print(category_columns)


[NumericColumn(key='I1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I10', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='I11', shape=(1,), default_val

In [87]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
  tf.keras.layers.DenseFeatures(numerical_columns+category_columns), # Placing the feature into tf.keras.layers.DenseFeatures
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['AUC'])

train_data = ds_train.shuffle(500)
test_data = ds_test

model.fit(train_data, epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
9/9 [==============================] - 0s 5ms/step - loss: 21.6733 - auc: 0.4658
Epoch 2/10
9/9 [==============================] - 0s 5ms/step - loss: 3.7753 - auc: 0.5934
Epoch 3/10
9/9 [==============================] - 0s 5ms/step - loss: 4.6664 - auc: 0.6029
Epoch 4/10
9/9 [==============================] - 0s 5ms/step - loss: 4.5184 - auc: 0.6053
Epoch 5/10
9/9 [==============================] - 0s 4ms/step - loss: 3.8879 - auc: 0.6184
Epoch 6/10
9/9 [==============================] - 0s 4ms/step - loss: 3.0287 - auc: 0.6270
Epoch 7/10
9/9 [==============================] - 0s 5ms/step - loss: 1.9252 - auc: 0.6454
Epoch 8/10
9/9 [==============================] - 0s 4ms/step - loss: 0.9001 - auc: 0.6635
Epoch 9/10
9/9 [==============================] - 0s 4ms/step - loss: 0.5685 - auc: 0.6898
Epoch 10/10
9/9 [==============================] - 0s 4ms/step - loss:

In [81]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test AUC {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
9/9 [==============================] - 0s 7ms/step - loss: 0.4357 - auc: 0.7876


Test Loss 0.43565914034843445, Test AUC 0.7876359224319458
